# Исследование надёжности заёмщиков

**Цели:**
- предварительно подготовить данные для анализа (избавиться от пропусков и дубликатов, обработать артефакты);
- выделить категории;
- ответить на вопросы: 
    - Влияет ли семейное положение и наличие детей на возврат кредита в срок? 
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок? 
    - Как разные цели кредита влияют на его возврат в срок?

**Задача:**
Анализ данных для последующего построения модели кредитного скоринга. Необходимо установить, какие факторы влияют на факт погашения кредита в срок.

**Данные:**
Данные о клиентах предоставлены банком и содержат:

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

**Использованные библиотеки:**
pandas

## Шаг 1. Общая информация

In [1]:
import pandas as pd 
clients = pd.read_csv('/datasets/data.csv')  # чтение файла с данными и сохранение в clients
clients.head(10)                             # получение первых 10 строк таблицы clients

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
clients.info() # получение общей информации о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:**
Итак, в таблице 12 столбцов, с разными типами данных: object, int64, float64. Бросаются в глаза следующие проблемы с данными:

- пропуски;
- неудачно выбранный формат отображения данных; 
- отрицательные значения в столбце `days_employed`;
- значения в столбце `education` не приведены к единообразию;
- в колонке `purpose` не хватает категоризации.

Чтобы двигаться дальше, необходима предобработка.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
clients.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Взглянем на строки с пропущенными значениями.

In [4]:
clients[clients['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Заметим, что они совпадают с пропусками в столбце `total_income`. Скорее всего, недостаток данных обусловлен пропусками в анкетах клиентов. Несмотря на это мы можем использовать их данные для исследования: информация о наличии долга, семейном статусе и количестве детей присутствует.

In [5]:
clients.agg({'days_employed': ['min', 'max'], 'total_income': ['min', 'max']}) 
# метод .agg позволит сделать это для каждого столбца

,days_employed,total_income
min,-18388.949901,2.066726e+04
max,401755.400475,2.265604e+06


Здесь становятся очевидны большие проблемы со столбцом `days_employed`: 
- отрицательные значения;
- неправдоподобные значения стажа (даже при стаже 50 лет значение не может превышать 50 х 365 = 18250);
- неверный формат отображения данных.

Постараемся исправить некоторые из эти проблем:

Отрицательные значения, очевидно, появились из-за неправильной записи (может использовалось тире?), то есть имел место человеческий фактор. Вместо них должны использоваться те же числа, но со знаком "+". Возьмем в столбце `days_employed` числа по модулю и проверим результат.

In [6]:
clients['days_employed'] = abs(clients['days_employed'])
clients.agg({'days_employed': ['min', 'max'], 'total_income': ['min', 'max']})

,days_employed,total_income
min,24.141633,2.066726e+04
max,401755.400475,2.265604e+06


In [7]:
days_employed_max = clients[clients['days_employed'] > 20000]
# возьмем все значения в столбце <days_employed> больше 20000 часов (т.е. более 50 лет)
len(days_employed_max)

3445

In [8]:
days_employed_max.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


Почти 3,5 тысячи строк заполнены артефактами, не считая пропусков и отрицательных значений. В рабочей ситуации необходимо сигнализировать коллегам о таких проблемах при сборе данных. Но в этом случае обратим внимание, что неправильный сбор данных коснулся в основном пенсионеров. Характер ошибки не ясен, но если разделить значения-артефакты на возраст клиентов, то получатся значения стажа, близкие к разумным. Прежде чем сделать это для всего столбца `days_employed`, проверим данные в столбце `dob_years`.

In [9]:
clients.agg({'dob_years': ['min', 'max']}) # проверим разброс значений

,dob_years
min,0
max,75


In [10]:
clients['dob_years'].loc[clients['dob_years'] == 0].count() # посчитаем количество нулевых значений

101

Более 100 человек не указали свой возраст. Чтобы заполнить пропуски, используем среднее значение, так как разброс между значениями не очень большой.

In [11]:
clients.loc[clients['dob_years'] == 0, 'dob_years'] = clients['dob_years'].mean()
clients.agg({'dob_years': ['min', 'max']}) # обратим внимание, что из-за наших действий столбец `dob_years` изменил тип данных,
# с этой проблемой мы справимся позже

,dob_years
min,19.0
max,75.0


In [12]:
for year in clients['days_employed']: #для каждого значения стажа в таблице
    if year > 20000:                                                            # для значений свыше 20000 часов
        age = clients.loc[clients.loc[:, 'days_employed'] == year]['dob_years'] # запишем возраст в переменную
        new_year = year / age                                                   # разделим стаж на возраст
        clients['days_employed'] =  clients['days_employed'].replace(year, new_year)  # заменим стаж в исходной таблице новым
clients.head() # убедимся, что все прошло успешно

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,6420.114567,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Столобец со стажем содержит много некорректных значений(слишком большие, отрицательные). И даже замена на медиану не решит ситуацию. Поскольку в дальнейшем нам этот столбец не нужен, то пропуски можно оставить как есть. Но обязательно сообщить разработчикам базы данных о подобной проблеме во избежании дальнейших неприятностей.
 
</div>

<a id='num_1'></a>
<div class="alert alert-danger"> Согласна, что заменять значения в столбцах `days_employed` и `total_income` неправильно. Делаю это исключительно для тренировки, но не могу придумать ничего лучше медианы:( Чтобы зарплаты разных категорий не так сильно сливались, пришла мысль вычислить медиану для той или иной категории отдельно.

In [13]:
for types in clients['income_type'].unique():    # для каждого типа занятости
    median = clients.loc[clients['income_type'] == types, 'total_income'].median() # посчитать медиану дохода, записать переменную
    clients.loc[(clients['total_income'].isna()) & (clients['income_type'] == types), 'total_income'] = median
    # в строках, где есть пропуск в столбце 'total_income' и указан этот тип занятости подставить переменную в 'total_income'

In [14]:
for years in clients['dob_years'].unique(): # аналогично рассчитаем медиану для 'days_employed' в зависимости от возраста
    median = clients.loc[clients['dob_years'] == years, 'days_employed'].median()
    clients.loc[(clients['days_employed'].isna()) & (clients['dob_years'] == years), 'days_employed'] = median
    # заполним пропуски по тому же принципу 

In [15]:
clients.isna().sum() # проверяем результат

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div class="alert alert-danger">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Хм, но при такой замене в тех сторках, где были пропуски, зарплата у госслжащего и пенсионера будет одинаковая. Как думаешь, как можно решить эту проблему?
    
 
</div>

**Вывод:**

<div class="alert alert-info"> Мы проверили наличие пропусков в датафрейме, установили закономерности в их появлении и выявили проблемы при сборе информации, большое количество артефактов и отрицательные значения. В реальных условиях исправлять их самостоятельно некорректно: необходимо привлекать коллег и искать причину. В учебных условиях отрицательные значения заменили положительными, артефакты привели к адекватным значениям, пропуски заполнили медианным или средним значением, поскольку имеем 

<div class="alert alert-success">
<h2> Комментарий от ревьюера 2<a class="tocSkip"></a></h2>
Конечно, уровень дохода сильно зависит от типа занятости и от возраста. Ты сделала все правильно, сгруппировав данные по какому-либо признаку. Кстати, в данном случае очень удобно воспользоваться методом <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html'>transform()</a>
</div>

### Замена типа данных

In [16]:
clients.info() # взглянем еще раз на тип данных в таблице, чтобы выявить проблемы

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


В столбце `days_employed` явно использован неверный тип данных float64, хотя для подсчета количества дней нужны целые числа. В столбце `total_income` такой тип данных возможен (например, при делении дохода за год на 12), но неудобен для визуального восприятия. А столбец `dob_years` - результат наших действий. Изменим тип данных во всех трех столбцах. Для этого выберем метод `astype`, потому что нам нужно перевести данные в формат 'int'.

In [17]:
def replace_wrong_types(columns, correct_type): # Создадим функцию для замены типов данных В качестве аргументов возьмем 
    for column in columns:                      # названия столбцов, где нужно произвести замену и нужный тип данных
        try:
            clients[column] = clients[column].astype(correct_type) # применим метод .astype() к столбцу и перезапишем его
        except:
            print('Ошибка в данных в столбце:', column) # в случае ошибки выведем название столбца, где ее искать
    
columns = ['days_employed', 'total_income', 'dob_years'] # список столбцов, где требуется замена
correct_type = 'int'                                     # нужный тип данных
replace_wrong_types(columns, correct_type)               # вызов функции
clients.info()                                           # проверяем результат

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**


<div class="alert alert-info"> В замене типа данных нуждались три столбца. Для этой цели используем функцию с методом *.astype()*. Он более универсален чем *to_numeric()*  и переводит данные в любой тип. Но необходимо предусмотреть возможность возникновения ошибки с помощью конструкции `try/except` (например, если появятся новые строки с пустыми значениями). В этом случае мы узнаем столбец, где она возникла.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Отлично, теперь все столбцы имеют корректный тип данных
    
 
</div>

### Обработка дубликатов

<div class="alert alert-danger"> Да, не подумала о том, что регистр влияет на подсчет дубликатов. Постаралась исправить. 
    <a id='num_2'></a>

[Возврат к ссылкам](#content)

In [18]:
clients['education'] = clients['education'].str.lower() # приведем столбцы со строками к единому регистру
clients['family_status'] = clients['family_status'].str.lower()

In [19]:
clients[clients.duplicated(keep=False)].sort_values(by = list(clients.columns)) # поиск явных дубликатов

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
15892,0,690,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
19321,0,690,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем
3452,0,1315,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
18328,0,1315,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости
4216,0,1420,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
9238,2,1615,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9013,2,1799,36,высшее,0,женат / замужем,0,F,госслужащий,0,150447,получение образования
14432,2,1799,36,высшее,0,женат / замужем,0,F,госслужащий,0,150447,получение образования
11033,2,1891,39,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу


In [20]:
clients = clients.drop_duplicates().reset_index(drop=True) # удалим явных дубликатов и перезапишем индексы

In [21]:
clients.info() # проверка: количество строк изменилось

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Возможно, дубликаты возникли потому, что одна и та же информация о клиенте поступала в датасет из нескольких источников. Такие дубликаты относятся к явным и легко удаляются методами pandas. Но есть еще неявные дубликаты. Их придется искать отдельно в каждом столбце. Для удобства добавим новые разделы к оглавлению.

<div class="alert alert-warning">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Здесь важно обратить внимание на то, что сначала необходимо привести текстовый формат данных к единому виду(регистр), а затем проверять на дубликаты. Иначе результат может получится другой.
 
</div>

<div class="alert alert-success">
<h2> Комментарий от ревьюера 2<a class="tocSkip"></a></h2>
👍
</div>

#### Столбец "children"

In [22]:
clients['children'].value_counts() # посчитаем количество уникальных значений в столбце

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Очевидно, что не бывает -1 ребенка, скорее всего это неправильно записанная единица. Двадцати детей тоже быть не может, особенно почти в 80 семьях. Предположим, что таким образом ошибочно были записаны двое детей. Их нужно считать вместе с аналогичными значениями. Исправим и проверим результат. 

In [23]:
clients['children'] = clients['children'].replace({20: 2, -1: 1}) # замена ошибочных значений на верные
clients['children'].value_counts() # проверка 

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

#### Столбцы "education" и "education_id"

In [24]:
clients.groupby('education')['education_id'].value_counts() # проверим как соотносятся различные значения образования 
# и их идентификаторы

education            education_id
высшее               0                5250
начальное            3                 282
неоконченное высшее  2                 744
среднее              1               15172
ученая степень       4                   6
Name: education_id, dtype: int64

Обращают на себя внимание три категории 2, 3, 4: они очень малочислены. Мы можем объединить "неоконченное высшее" и "начальное образование", так как под последним, очевидно, имеется ввиду законченное школьное обучение (в России этот этап обязателен), а "неоконченное высшее" всего на несколько лет отличается от него. А вот "ученая степень" может быть отнесена к высшему образованию. Так категории станут компактнее и проще поддадутся анализу. При замене нужно учесть сбой идентификаторов из соседнего столбца `education_id`.

In [25]:
clients['education'] = clients['education'].replace({'начальное': 'неоконченное высшее', 'ученая степень': 'высшее'})
clients['education_id'] = clients['education_id'].replace({3: 2, 4: 0})

In [26]:
clients['education'].value_counts() 

среднее                15172
высшее                  5256
неоконченное высшее     1026
Name: education, dtype: int64

#### Столбцы "family_status" и "family_status_id"

In [27]:
clients.groupby('family_status')['family_status_id'].value_counts() # посчитаем разнообразие вариантов в столбце `family_status`

family_status          family_status_id
в разводе              3                    1195
вдовец / вдова         2                     959
гражданский брак       1                    4151
женат / замужем        0                   12339
не женат / не замужем  4                    2810
Name: family_status_id, dtype: int64

Проблем в столбце не обнаружено.

#### Столбец "gender"

In [28]:
clients['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

Как выяснилось, в наших данных существует еще один пропуск, а именно не указан пол одного из клиентов. В данном исследовании пол клиента не играет значения, поэтому не станем присваивать здесь какое-то определенное значение. Лучше заменим непонятное сокращение на "unknown".

In [29]:
clients['gender'] = clients['gender'].replace('XNA', 'unknown')

#### Столбец "income_type"

In [30]:
clients['income_type'].value_counts() # посчитаем все уникальные значения в этом столбце и увидим, что некоторые 
# из них очень малочислены, т. е. могут быть объединены с другими для компактности.

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [31]:
clients['income_type'] = clients['income_type'].replace({'предприниматель': 'компаньон', 'студент': 'безработный', 'в декрете': 'сотрудник'}) 
# замена ошибочных значений на верные
clients['income_type'].value_counts()

сотрудник      11085
компаньон       5080
пенсионер       3829
госслужащий     1457
безработный        3
Name: income_type, dtype: int64

Предпринимателей объединим с компаньонами в бизнесе, студента отнесем к безработным, клиента в декрете - к сотрудникам, поскольку он/она все еще является сотрудником той или иной организации. Объединить всех безработных с другой категорией (например, пенсионерами) невозможно, так как будет наблюдаться разница в возрасте среди клиентов. Кроме того, у безработного может появиться работа, тогда категория изменится.

#### Столбец "debt"

In [32]:
clients['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

Здесь никаких проблем не обнаружено. В столбце только два значения, обозначающих была или нет задолженность по кредиту.

**Вывод**

<div class="alert alert-info"> В первую очередь выявили и удалили явные дубликаты. Неявные дубликаты пришлось искать вручную в каждом столбце отдельно, поскольку в них содержится разная информация. В некоторых столбцах были исправлены ошибки при вводе данных, унифицирован регистр, заполнены пропуски, незамеченные ранее, а также удалены лишние категории. Для этих целей были использованы подсчет уникальных значений, замена, приведение к единому регистру для строк. 
    
По результатам данного этапа можно обосновать необходимость категоризации данных, так как разделять на отдельные категории людей, имеющих 3 и более детей нецелесообразно. То же самое касается столбца с информацией об образовании и целях кредита.
В последнем даже невооруженным взглядом видно, что многие цели дублируют друг друга, и необходима предварительная лемматизация, чтобы привести все данные к единообразию. 

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Супер, данные корректны, можно приступать к следующей части исследования)
</div>

### Лемматизация

In [33]:
clients['purpose'].value_counts() # выберем все уникальные значения в столбце `purpose` и посчитаем их

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [34]:
purposes_list = ' '.join(clients['purpose'].unique()) # соединим их в одну строку,которую потом можно лемматизировать
from pymystem3 import Mystem # получение лемматизатора
m = Mystem()
lemmas = m.lemmatize(purposes_list) # лемматизация списка `purposes_list`

from collections import Counter # изпользуем коллекцию Counter для подсчета количества лемм
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Опираясь на данные лемматизации, можно выделить следующие категории, к которым относятся все значения в столбце `purpose`:
- недвижимость;
- автомобиль;
- образование;
- свадьба;
- ремонт.

Необходимость ввести отдельную категорию для ремонта жилья можно обосновать тем, что эти расходы несопоставимы с суммами на покупку или строительство недвижимости. Следовательно, условия выдачи такого кредита могут рассчитываться иначе, чем для других категорий, связаных с жильем, и  возвращать такой кредит, возможно, будут иначе. Количество клиентов, берущих кредит на ремонт (612 человек), также не позволяет просто отбросить эти данные или соединить с другой категорией

In [35]:
def purposes_grouped(row): # создадим функцию для определения идентификаторов к столбцу `purpose`, которая лемматизирует ячейку
    item = m.lemmatize(row) # сохранит результат в переменной
    try:
        if 'недвижимость' in item or 'жилье' in item: # и в зависимости от обнаруженных в переменной лемм будет присваивать
            if 'ремонт' in item:                      # строке тот или иной идентификатор
                return 'ремонт'
            return 'недвижимость'
        if 'автомобиль' in item:
            return 'автомобиль'
        if 'образование' in item:
            return 'образование'
        if 'свадьба' in item:
            return 'свадьба'
    except:
        return 'без категории'         # на случай, если появятся новые значения, которые функция не сумеет обработать   

In [36]:
clients['purposes_grouped'] = clients['purpose'].apply(purposes_grouped) # с помощью метода *.apply()* добавим новый столбец 
clients.head() # с категориями к датафрейму и для проверки посмотрим на несколько строк обновленного датафрейма

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purposes_grouped
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,6420,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

<div class="alert alert-info"> С помощью лемматизации удалось выделить 5 категорий, которые охватывают все возможные цели кредита из датафрейма. Принято решение обозначать их именно строками, так как человеку их проще воспринимать, чем цифры, а нам придется презентовать результаты анализа. 
    Введение единых категорий упрощает заполнение датафрейма и помогает ответить на вопросы исследования о зависимости цели кредита и сроков его возврата. Для других столбцов также необходимо провести категоризацию.
    <div

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Лемматизация прошла успешно. Удалось выделить основные цели получения кредита
</div>

### Категоризация данных

Так же как мы категоризировали цели кредита, необходимо поступить со столбцами `children`, `dob_years` и `total_income`, так как они необходимы нам для ответа на главные вопросы исследования. 

В столбце `children` очевиден большой разрыв между бездетными, теми, кто имеет 1-2 детей, и многодетными, то есть объединить их проще, чем учитывать каждую величину. 

In [37]:
def children_grouped(children): # напишем функцию, которая принимает в качестве аргумента количество детей у клиента
    try:
        if children == 0:      # и в зависимости от этого числа возвращает то или иное название категории
            return 'нет детей'
        if 1 <= children <= 2:
            return '1-2 ребенка'
        if 3 <= children <= 9:
            return 'многодетные'
    except:                          # на случай появления пустых или ошибочных значений предусмотрим вывод-предупреждение
        return 'Ошибочное значение'
    
clients['children_grouped'] = clients['children'].apply(children_grouped) # добавим столбец с новыми данными к датафрейму
clients['children_grouped'].value_counts() # и подсчитаем количество клиентов в каждой из новых категорий

нет детей      14091
1-2 ребенка     6983
многодетные      380
Name: children_grouped, dtype: int64

<a id='num_3'></a>
Аналогичным образом поступим со столбцом `total_income`: здесь представлено большое разнообразие значений, которые удобнее отнести к тому или иному диапазону.
[Возврат к ссылкам](#content)

In [38]:
pd.qcut(clients['total_income'], q=3) # с помощью метода *qcut()* выясним диапазоны, в которые укладываются значения из столбца

0        (172357.0, 2265604.0]
1        (20666.999, 119261.0]
2         (119261.0, 172357.0]
3        (172357.0, 2265604.0]
4         (119261.0, 172357.0]
                 ...          
21449    (172357.0, 2265604.0]
21450     (119261.0, 172357.0]
21451    (20666.999, 119261.0]
21452    (172357.0, 2265604.0]
21453    (20666.999, 119261.0]
Name: total_income, Length: 21454, dtype: category
Categories (3, interval[float64]): [(20666.999, 119261.0] < (119261.0, 172357.0] < (172357.0, 2265604.0]]

In [39]:
def income_grouped(income): # напишем функцию, которая принимает в качестве аргумента ежемесячный доход клиента
    try:
        if income < 120000: # и в зависимости от этого возвращает то или иное название категории
            return 'низкий'
        if 120000 <= income <= 170000:
            return 'средний'
        if income > 170000:
            return 'высокий'
    except:                             # страховка на случай появления пустых или ошибочных значений
        return 'Неизвестное значение'
    
clients['income_grouped'] = clients['total_income'].apply(income_grouped) # добавим столбец с новыми данными к датафрейму
clients['income_grouped'].value_counts() # и подсчитаем количество клиентов в каждой из новых категорий

высокий    7718
низкий     7231
средний    6505
Name: income_grouped, dtype: int64

<div class="alert alert-danger">Так категории получились более равномерными. Спасибо за подсказку с методом, не знала о таком. Откорректировала вывод в связи с новыми данными.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
А почему ты выбрала именно такие пограницные значения? Здесь очень удобно воспользоваться методом quantile() или методом qcut(), чтобы разделить данные на равные группы
</div>

Введем новые категории и для значений возраста в столбце `dob_years`:

In [40]:
def age_grouped(age): # напишем функцию, которая принимает в качестве аргумента ежемесячный доход клиента
    try:
        if 18 <= age <= 24: # и в зависимости от этого возвращает то или иное название категории
            return 'молодежь'  # молодые люди, только что закончившие обучение или едва начавшие работать
        if 25 <= age <= 64:
            return 'взрослые' # люди, которые профессионально активны
        if age >= 65:
            return 'пенсионеры' # люди пенсионного возраста
    except:                             # страховка на случай появления пустых или ошибочных значений
        return 'Неизвестное значение'
    
clients['age_grouped'] = clients['dob_years'].apply(age_grouped) # добавим столбец с новыми данными к датафрейму
clients['age_grouped'].value_counts() # и подсчитаем количество клиентов в каждой из новых категорий

взрослые      19684
пенсионеры      895
молодежь        875
Name: age_grouped, dtype: int64

**Вывод**

<div class="alert alert-info"> Таким образом, для пользы исследования разделим клиентов на категории по возрасту, уровню дохода и количеству детей. Сразу же можно увидеть, что основная масса клиентов банка это люди от 25 до 64 лет без детей с высоким (более 170 000) или низким (менее 120 000) уровнем дохода в месяц. Тех, кто имеет 1-2 детей в два раза меньше, чем бездетных. Меньше всех в датафрейме представлены молодежь, пенсионеры и многодетные семьи. Эти данные необходимо учесть на следующей стадии анализа.
    
    
Здесь заканчивается этап предобработки данных. Даже если все обработанные столбцы не потребуются нам в дальнейшем для исследования, привести их в порядок необходимо на тот случай, если с данными будет вестись работа в будущем. Возможно, потребуется дать ответы на другие вопросы или уточнить информацию. Далее необходимо выбрать те параметры, которые дадут наиболее полные ответы на поставленные вопросы, произвести необходимые расчеты, представить результаты и описать выводы. 

## Шаг 3. Ответьте на вопросы

Для удобства создадим новый датафрейм, с нужными нам для исследования колонками. Это поможет быстрее ориентироваться в данных и не портить изменениями исходную таблицу.

In [41]:
clients_compact = clients[['gender', 'age_grouped', 'family_status', 'children_grouped', 'education', 'income_grouped', 'debt', 'purposes_grouped']]
clients_compact.head()

,gender,age_grouped,family_status,children_grouped,education,income_grouped,debt,purposes_grouped
0,F,взрослые,женат / замужем,1-2 ребенка,высшее,высокий,0,недвижимость
1,F,взрослые,женат / замужем,1-2 ребенка,среднее,низкий,0,автомобиль
2,M,взрослые,женат / замужем,нет детей,среднее,средний,0,недвижимость
3,M,взрослые,женат / замужем,многодетные,среднее,высокий,0,образование
4,F,взрослые,гражданский брак,нет детей,среднее,средний,0,свадьба


Сначала ответим на основные вопросы проекта, создав универсальную функцию для этого:

In [42]:
def relation (column, category):       # создадим функцию, которая в качестве аргумента принимает столбец и категорию из него 
    quantity = clients_compact[clients_compact[column] == category]
    # выберем строки, которые содержат инф-цию о клиентах в этой категории,
    debtors = quantity[quantity['debt'] == 1]          # из этих строк выберем клиентов с задолженностью
    debt_quantity = len(debtors) / len(quantity)       # посчитаем соотношение должников в этой категории к общему числу 
    return '{:.1%}'.format(debt_quantity)              # клиентов в этой категории              
# функция возвращает соотношение, выраженное в процентах

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Очень круто, что ты пишешь функции. Так код становится более структурированным
</div>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
# создадим небольшую таблицу, которая наглядно представит полученный результат
data = [
    ['нет детей', relation('children_grouped', 'нет детей')],    # функция relation заполнит строки таблицы,
    ['1-2 ребенка', relation('children_grouped', '1-2 ребенка')],
    ['многодетные', relation('children_grouped', 'многодетные')]    
]                                                                    
columns = ['children_grouped', 'debtors']                 # колонки - это категории клиентов с детьми и количество должников
children_debt = pd.DataFrame(data = data, columns = columns)   
children_debt.sort_values(by = 'debtors', ascending = False)   # результат отсортируем по убыванию количества должников

,children_grouped,debtors
1,1-2 ребенка,9.3%
2,многодетные,8.2%
0,нет детей,7.5%


**Вывод**

<div class="alert alert-info"> Согласно полученным данным просматривается очевидная зависимость между увеличением количества детей и появлением задолженностей по кредиту. Клиенты с 1-2 детьми в 9,3% случаев имеют проблемы со своевременным возвратом долга. От них немного отличаются в лучшую сторону многодетные родители, которые имеют задолженности по кредиту в 8,2% случаев, но их в целом очень мало в общем количестве клиентов. Лучше всего ситуация обстоит с клиентами, которые не имеют детей: они не соблюдают срок возврата кредита всего лишь в 7,5% случаев и чаще всего обращаются за кредитом. 

<div class="alert alert-success">
<h2> Комментарий от ревьюера <a class="tocSkip"></a></h2>
Верно:) Да, заемщики без детей самые ответственные.
    
 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
# создадим небольшую таблицу, которая наглядно представит полученный результат
data = [
    ['женат / замужем', relation('family_status', 'женат / замужем')],  # функция relation заполнит строки таблицы,
    ['гражданский брак', relation('family_status', 'гражданский брак')], 
    ['вдовец / вдова', relation('family_status', 'вдовец / вдова')],      
    ['в разводе', relation('family_status', 'в разводе')],                
    ['не женат / не замужем', relation('family_status', 'не женат / не замужем')]
]
columns = ['family_status', 'debtors']                  # колонки - это виды семейного положения и количество должников
family_debt = pd.DataFrame(data = data, columns = columns)     
family_debt.sort_values(by = 'debtors', ascending = False)     # результат отсортируем по убыванию количества должников

,family_status,debtors
4,не женат / не замужем,9.8%
1,гражданский брак,9.3%
0,женат / замужем,7.5%
3,в разводе,7.1%
2,вдовец / вдова,6.6%


In [45]:
family_pivot = clients.pivot_table(index = ['children_grouped', 'debt'], columns = 'family_status', values = 'family_status_id', aggfunc = 'count')
family_pivot

family_status          в разводе  вдовец / вдова  гражданский брак  \
children_grouped debt                                                
1-2 ребенка      0         370.0            95.0            1204.0   
                 1          29.0            10.0             151.0   
многодетные      0          11.0             7.0              58.0   
                 1           1.0             NaN               8.0   
нет детей        0         729.0           794.0            2501.0   
                 1          55.0            53.0             229.0   

family_status          женат / замужем  не женат / не замужем  
children_grouped debt                                          
1-2 ребенка      0              4191.0                  476.0  
                 1               395.0                   62.0  
многодетные      0               265.0                    8.0  
                 1                20.0                    2.0  
нет детей        0              6952.0                 2052.0  
                 1               516.0                  210.0

**Вывод**

<div class="alert alert-info"> Наблюдается зависимость между стабильностью семейного положения, количеством детей и своевременным возвратом заемных средств. Наиболее рискованными категориями заемщиков являются неженатые/незамужние люди, а также те, кто считает себя состоящими в гражданском браке. Такие люди чаще берут кредиты и реже их возвращают. Возможно, это связано с гибкостью их статуса: они легче подвержены изменениям (переездам, смене места работы или приоритетов). Тогда как семейные пары отличаются большей стабильностью: они тоже часто берут кредиты, но и возращают их в срок тоже чаще. Самыми обязательными оказались бездетные вдовцы: среди них должников всего 6,6%.

<div class="alert alert-success">
<h2> Комментарий от ревьюера <a class="tocSkip"></a></h2>
Посмотрев на сводную таблицу, можно увидеть, что доля просроченых кредитов больше у людей, которые находятся в гражданском браке или не женаты. Чаще всего возращают кредиты вдовцы и вдовы.
</div>


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a id='num_4'></a>
[Возврат к ссылкам](#content)

In [46]:
# снова создадим таблицу для наглядности по схожим принципам
data = [
    ['низкий', relation('income_grouped', 'низкий')],  
    ['средний', relation('income_grouped', 'средний')], 
    ['высокий', relation('income_grouped', 'высокий')]
]
columns = ['income_grouped', 'debtors'] 
income_debt = pd.DataFrame(data = data, columns = columns)     
income_debt.sort_values(by = 'debtors', ascending = False)  

,income_grouped,debtors
1,средний,8.9%
0,низкий,8.1%
2,высокий,7.5%


**Вывод**

<div class="alert alert-info"> Очевидно, что люди с высоким доходом возвращают кредиты в срок чаще (92,5%). Чаще всего пренебрегают обязательствами клиенты со средним доходом: они задерживают выплаты в 8,9% случаев.

<div class="alert alert-success">
<h2> Комментарий от ревьюера <a class="tocSkip"></a></h2>
Здесь многое зависит от того, как ты разобьешь данные на группы. Попробуй разделить на равные по количеству заемщиков группы и сравни результат)
</div>


- Как разные цели кредита влияют на его возврат в срок?

In [47]:
data = [
    ['недвижимость', relation('purposes_grouped', 'недвижимость')],  
    ['автомобиль', relation('purposes_grouped', 'автомобиль')], 
    ['образование', relation('purposes_grouped', 'образование')],
    ['свадьба', relation('purposes_grouped', 'свадьба')],
    ['ремонт', relation('purposes_grouped', 'ремонт')]
]
columns = ['purposes_grouped', 'debtors'] 
purposes_debt = pd.DataFrame(data = data, columns = columns)     
purposes_debt.sort_values(by = 'debtors', ascending = False)

,purposes_grouped,debtors
1,автомобиль,9.4%
2,образование,9.2%
3,свадьба,8.0%
0,недвижимость,7.3%
4,ремонт,5.8%


**Вывод**

<div class="alert alert-info"> По полученным данным прослеживается связь между целями кредита и вероятностью его возврата в срок. Самыми рискованными с этой точки зрения являются такие цели как покупка автомобиля и получение образования (9,4% и 9,2% задолженностей. Возможно, это связано с тем, что страх расстаться с автомобилем в случае отказа выплачивать кредит в срок меньше, чем при покупке недвижимости (здесь 92,7% предпочитают не рисковать). А такие события как получение диплома и свадьба уже не получится отменить, даже при образовании задолженности, поэтому чувство ответственности ослабевает. Зато небольшие цели (например, ремонт) имеют самый высокий шанс (96,2%) окупиться в срок. 

Проведем еще несколько вычислений, чтобы найти другие интересные закономерности в датасете.

In [48]:
data = [                                             # зависимость своевременного возврата кредита от уровня образования
    ['среднее', relation('education', 'среднее')],  
    ['высшее', relation('education', 'высшее')], 
    ['неоконченное высшее', relation('education', 'неоконченное высшее')]
]
columns = ['education', 'debtors'] 
education_debt = pd.DataFrame(data = data, columns = columns)     
education_debt.sort_values(by = 'debtors', ascending = False)  

,education,debtors
2,неоконченное высшее,9.6%
0,среднее,9.0%
1,высшее,5.3%


**Вывод**

<div class="alert alert-info"> Налицо связь кредитных обязательств с уровнем образования. Чем он выше, тем выше вероятность вернуть долг вовремя, и наоборот. 

In [49]:
data = [                                             # зависимость своевременного возврата кредита от пола
    ['M', relation('gender', 'M')],  
    ['F', relation('gender', 'F')]
]
columns = ['gender', 'debtors'] 
gender_debt = pd.DataFrame(data = data, columns = columns)     
gender_debt.sort_values(by = 'debtors', ascending = False)  

,gender,debtors
1,F,7.0%
0,M,10.3%


**Вывод**

<div class="alert alert-info"> Также можем сделать вывод, что женщины относятся к возврату долга ответственнее, чем мужчины.

## Шаг 4. Общий вывод

# В ходе исследования мы проверили несколько гипотез и установили следующее:
- существует устойчивая связь между наличием детей и своевременной выплатой кредита: даже 1 или 2 ребёнка в семье увеличивают риск просрочек;
- семейные обязательства положительно сказываются на возврате кредита в срок: семейные люди и вдовцы - самые аккуратные плательщики, особенно если у них нет детей, тогда как неженатые и состоящие в гражданском браке чаще всех нарушают обязательства;
- уровень дохода ожидаемо связан со своевременностью выплат: люди с высоким доходом (от 170 000 рублей) имеют меньше долгов, люди со средним доходом (от 120 000 рублей) часто задерживают выплаты;
- погашение кредита в срок зависит и от цели заемщика: аккуратнее всего клиенты возвращают долги за недвижимость, а также небольшие суммы на текущие нужды (например, ремонт), а цели, теряющие свою ценность по мере получения (автомобиль), или чей результат не зависит от наличия долгов (образование, свадьба) - самые рискованные.

Помимо ответов на основные вопросы исследования, удалось установить следующие закономерности:
- возврат кредита связан и с образованием: чем выше его уровень, тем меньше шанс образования задолженности;
- женщины обращаются за кредитом чаще, но и возвращают долги ответственнее, чем мужчины.

По собранным данным видно, что чаще всего за кредитом обращаются женатые люди от 25 до 64 лет без детей. У них высоки шансы вернуть кредит в срок.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>

Поздравляю с первым успешным  проектом!

- Ты хорошо делаешь выводы и анализируешь результаты.
    
- Код написан аккуратно и качественно, видно, что есть какой-то определенный бэкграунд.

- Соблюдена структура проекта, было приятно проверять:)
    
- Все написано четко и по делу

 
Осталось исправить замечание и проект будет принят)

Желаю дальнейших успехов!


</div>

<div class="alert alert-info">
    Спасибо за добрые слова и быструю проверку!

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.